In [26]:
%cd ~/dissertation_code 
import sys
sys.path.append("RQ 3/")
import pandas as pd
import numpy as np


/home/tntech.edu/kebrown46/dissertation_code


In [27]:
model = "NN-dropout"
dataset = "trauma_uk"
implementation = "alpha"
res_key=f"dissertation_DT_noProb_{implementation}_newtest"
n_bins=3
df = f"RQ 3/results/{res_key}/subgroup_statistics_{model}_DT_{dataset}.csv"
df = pd.read_csv(df)
iteration = 0
# display(df)

In [28]:
tree_results = df[(df["no_bins"] == n_bins) & (df["iteration"] == 0)]

In [29]:
display(tree_results)
# display(list(tree_results))

,Unnamed: 0,model,dataset,iteration,test bin assignment accuracy,no_rules,max_depth,max_features,no_bins,subgroup 0 test bin assignment accuracy,...,subgroup 9 test variables used,subgroup 9 train bin assignment accuracy,subgroup 9 train classification accuracy,subgroup 9 train coverage,subgroup 9 train no. total variables,subgroup 9 train no. unique variables,subgroup 9 train predicted bin distribution,subgroup 9 train size,subgroup 9 train true bin distribution,subgroup 9 train variables used
0,0,NN-dropout,trauma_uk,0,0.860257,39,9,32,2,0.926471,...,"['AMPT', 'AGE', 'CHESTAMPT', 'HEADAMPT', 'Mech...",0.689202,0.889202,0.020801,7,7,Counter({0: 1065}),1065,"Counter({0: 734, 1: 331})","['AMPT', 'AGE', 'CHESTAMPT', 'HEADAMPT', 'Mech..."


In [57]:

new_columns = ["subgroup no.", 
              "test bin assignment accuracy", 
              "test classification accuracy", 
              "test coverage", 
              "test predicted no. 0s", 
              "test predicted no. 1s", 
              "test predicted no. 2s",
              "test size",
              "test true no. 0s",
              "test true no. 1s",
              "test true no. 2s",
              "test no. unique variables",
              "test no. total variables",
               "test variables used"
              ]


_retrieve_value_columns = set(new_columns.copy()) - set(["subgroup no.", "test predicted no. 0s", "test predicted no. 1s","test predicted no. 2s",  "test true no. 0s", "test true no. 1s", "test true no. 2s"])

print(_retrieve_value_columns)

def _convert_df_bin_count_to_bin_predicted(count_string):
    #count_string = "Counter({0:X, 1:Y})"
    # print(count_string)
    res = {0:0, 1:0, 2:0}
    # print(count_string)
    count_string = count_string[9:len(count_string)-2]
    counts = count_string.split(",")
    for str_i in counts:
        str_i = str_i.split(":")
        try:
            i = int(str_i[0])
            v = int(str_i[1])
            res[i] = v
        except:
            continue
    return res


def _get_distribution(row, subgroup_no, og_col_name, new_name):
    cs = row[f"subgroup {subgroup_no} {og_col_name}"].values[0]
    count = _convert_df_bin_count_to_bin_predicted(cs)
    to_return = {}
    for key in count:
        to_return[f"{new_name} {key}"] = count[key]
    return to_return

def _retrieve_value(row, subgroup_no, col):
    return row[f"subgroup {subgroup_no} {col}"].values[0]

def get_row(subgroup, tree_results):
    to_return = {"subgroup no.": subgroup}
    for column in _retrieve_value_columns:
        to_return[column] =  _retrieve_value(tree_results, subgroup, column)
    to_return.update(_get_distribution(tree_results, subgroup, "test predicted bin distribution", "test predicted no."))
    to_return.update(_get_distribution(tree_results, subgroup, "test true bin distribution", "test true no."))
    
    return to_return
        
def create_dataframe(tree_results):
    num_subgroups = tree_results["no_rules"].values[0]
    to_return = []
    for i in range(num_subgroups):
        to_return.append(get_row(i, tree_results))
    to_return =  pd.DataFrame(to_return)
    column_order = ["subgroup no.", 
                    "test bin assignment accuracy", 
                    "test classification accuracy", 
                    "test coverage", 
                    "test size",
                    "test predicted no. 0", 
                    "test predicted no. 1", 
                    "test predicted no. 2", 
                    "test true no. 0",
                    "test true no. 1",
                   "test true no. 2",
                   "test no. unique variables",
                  "test no. total variables",
                   "test variables used"]
    return to_return[column_order]
        
def calculate_metrics(results):
    acc = results["test bin assignment accuracy"].values
    cov = results["test coverage"].values
    n_unique = results["test no. unique variables"].values
    n_total = results["test no. total variables"].values
    max_features = tree_results["max_features"].values
    ni = 0.2 * max_features
    
    results["rule_effectiveness"] = acc * cov
    results["rule_simp_comp"] = n_unique/n_total
    results["exp_no_interaction"] = (0.5/(0.2*max_features)) + (0.5/n_unique)
    results["exp + effect"] = (0.25/(max_features)) + (0.25/n_unique) + (0.5 * (acc*cov))
    # display(results)
    # #exp_interaction
    return results

def print_mean_results(results, n_bins, dataset, model, implementation):
    print(f"{n_bins}_{dataset}_{model}_{implementation}")
    RE = np.mean(results["rule_effectiveness"].values)
    REprime = RE * results.shape[0]
    RS = np.mean(results["rule_simp_comp"].values)
    E1 = np.mean(results["exp_no_interaction"].values)
    E2 = np.mean(results["exp + effect"].values) * results.shape[0]
    
    print(f"RE = {RE}")
    print(f"REprime = {REprime}")
    print(f"RS = {RS}")
    print(f"E1 = {E1}")
    print(f"E2 = {E2}")

def calculate_results(model, dataset, implementation, n_bins):
    res_key=f"dissertation_DT_noProb_{implementation}_newtest"
    df = f"RQ 3/results/{res_key}/subgroup_statistics_{model}_DT_{dataset}.csv"
    df = pd.read_csv(df)
    iteration = 0
    tree_results = df[(df["no_bins"] == n_bins) & (df["iteration"] == 0)]
    results = create_dataframe(tree_results)   
    results = calculate_metrics(results)
    results.to_csv(f"RQ 3/results/{res_key}/case_study_nbins{n_bins}_{dataset}_{model}_{iteration}.csv")
    print_mean_results(results, n_bins, dataset, model, implementation)

{'test no. unique variables', 'test bin assignment accuracy', 'test size', 'test classification accuracy', 'test coverage', 'test no. total variables', 'test variables used'}


In [58]:

calculate_results("NN-dropout", "trauma_uk", "alpha", 2)
calculate_results("NN-dropout", "trauma_uk", "alpha", 3)
calculate_results("NN-dropout", "trauma_uk", "maxleaves", 2)
calculate_results("NN-dropout", "trauma_uk", "knee", 2)

2_trauma_uk_NN-dropout_alpha
RE = 0.02205786245160472
REprime = 0.860256635612584
RS = 0.8201058201058202
E1 = 0.1947916666666667
E2 = 3.0098158178062926
3_trauma_uk_NN-dropout_alpha
RE = 0.027661569109978284
REprime = 0.8298470732993485
RS = 0.7725000000000002
E1 = 0.2011805555555556
E2 = 2.4951318699830067
2_trauma_uk_NN-dropout_maxleaves
RE = 0.013875107026009402
REprime = 0.8602566356125829
RS = 0.8148669878508588
E1 = 0.1721630184331797
E2 = 3.8296818892348625
2_trauma_uk_NN-dropout_knee
RE = 0.40015820003515556
REprime = 0.8003164000703111
RS = 1.0
E1 = 0.578125
E2 = 0.9157832000351556


In [60]:
calculate_results("catboost-ve", "trauma_uk", "alpha", 2)
calculate_results("catboost-ve", "trauma_uk", "alpha", 3)
calculate_results("catboost-ve", "trauma_uk", "maxleaves", 2)
calculate_results("catboost-ve", "trauma_uk", "knee", 2)

2_trauma_uk_catboost-ve_alpha
RE = 0.021814027069783755
REprime = 0.8725610827913501
RS = 0.9428751803751805
E1 = 0.16993055555555553
E2 = 2.584891652506786
3_trauma_uk_catboost-ve_alpha
RE = 0.027186969004511584
REprime = 0.8156090701353476
RS = 0.8575132275132273
E1 = 0.1929265873015873
E2 = 2.3642033445914836
2_trauma_uk_catboost-ve_maxleaves
RE = 0.015864746959842715
REprime = 0.8725610827913494
RS = 0.9554184704184704
E1 = 0.15972902728584548
E2 = 3.110078791756425
2_trauma_uk_catboost-ve_knee
RE = 0.1402706978379328
REprime = 0.8416241870275969
RS = 1.0
E1 = 0.2725694444444444
E2 = 1.0510204268471317
